<a href="https://colab.research.google.com/github/PhoenixAlpha23/Finetune-gemma/blob/main/gemma_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aryanrahultandon_multilingual_idioms_indian_path = kagglehub.dataset_download('aryanrahultandon/multilingual-idioms-indian')
keras_gemma2_keras_gemma2_instruct_2b_en_1_path = kagglehub.model_download('keras/gemma2/Keras/gemma2_instruct_2b_en/1')

print('Data source import complete.')


## Fine-tuning to train gemma pretrained model on idioms from various languages

In [ ]:
import numpy as np
import pandas as pd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd

data= pd.read_json('/kaggle/input/multilingual-idioms-indian/combined_idioms.json')
data.head()

In [ ]:
!pip install transformers datasets bitsandbytes accelerate peft

## Data Preprocessing

In [ ]:
from datasets import load_dataset

# Load dataset from the Kaggle dataset path
data = load_dataset('json', data_files='/kaggle/input/multilingual-idioms-indian/combined_idioms.json')

# Preprocess dataset
data = data['train'].map(lambda x: {"input_text": x["idiom"], "target_text": x["figurative_meaning"]})
data = data.rename_column("input_text", "input_ids")
data = data.rename_column("target_text", "labels")
data = data.train_test_split(test_size=0.1)

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

In [ ]:
import os

# Set the backbend before importing Keras
os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

import keras_nlp
import keras

# Run at half precision.
#keras.config.set_floatx("bfloat16")

# Training Configurations
token_limit = 256
num_data_limit = 100
lora_name = "cakeboss"
lora_rank = 4
lr_value = 1e-4
train_epoch = 20
model_id = "gemma2_instruct_2b_en"

In [ ]:
import keras
import keras_nlp

import time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

tick_start = 0

def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")

def text_gen(prompt):
    tick()
    input = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    output = gemma_lm.generate(input, max_length=token_limit)
    print("\nGemma output:")
    print(output)
    tock()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gemma_causal_lm_preprocessor"  # Replace with GEMMA 2B model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)